# Material compute demo agent




## 目录与步骤指南
**该 notebook 仅供展示，如需运行请访问 github 仓库 https://github.com/dptech-corp/material-compute-agent**

本 Notebook 展示了一个创新的材料计算代理系统，它利用大型语言模型（LLM）和 CAMEL 多代理框架自动化执行材料科学中的计算任务。系统模拟了真实研究团队的协作过程，包括从文献中提取信息、生成计算输入文件、执行计算、分析结果并生成报告。

该系统特别适用于高通量材料筛选和计算材料科学研究，能够显著提高材料发现和优化的效率。通过集成 Bohrium 计算平台和 VASP 计算软件，系统可以执行从密度泛函理论（DFT）计算到热力学性质分析的各种任务。

本演示特别关注钙钛矿氧化物材料（如 Sr5Ca3Fe8O24）的计算模拟，这类材料在热化学能量存储（TCES）等领域具有重要应用前景。

1. **环境配置**
   - 安装依赖包 (`%pip install -e .`)
   - 下载并配置 MP 数据集
   - 设置 API 密钥 (Bohrium, OpenAI/Azure, DeepSeek)

2. **系统初始化**
   - 导入必要的库和工具
   - 创建模型实例
   - 配置代理角色和工具

3. **任务执行**
   - 启动 MCP 服务 (`python CalculationMCPServer.py`)
   - 连接到 MCP 服务器
   - 创建并配置工作流程
   - 提交计算任务

4. **输入示例**
   - 文章路径: test.pdf
   - 材料体系: Sr5Ca3Fe8O24
   - 参数修改: INCAR 的 NELM = 2


## 配置环境





### 安装依赖

In [ ]:
%pip install -e .

### 安装SDK

In [ ]:
!git clone -b lh https://github.com/dptech-corp/science-agent-sdk.git
%pip install -e ./science-agent-sdk

### 配置本地mp数据集，用于查询模板

In [ ]:
!wget https://openfiles.mlops.dp.tech/projects/lp/6583cfd69e8f402898b60830889d242d/mp_materials_cif.zip

!unzip -q mp_materials_cif.zip

### 引入依赖

In [1]:
import os
import warnings
import subprocess

from camel.toolkits.mcp_toolkit import MCPToolkit
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.agents import ChatAgent
from camel.tasks.task import Task
from camel.toolkits import (
    HumanToolkit
)
from camel.toolkits import FunctionTool
from camel.messages.base import BaseMessage

from vasp_function import read_vasp_pdf,write_vasp_report,analyze_vasprun_all,search_poscar_template,write_poscar,write_vasp_config,generate_vasp_config
from vasp_function import show_vasp_config,rewrite_vasp_config



from DPWorkforce.CalculationWorkforce import CalculationWorkforce
from dp.agent.adapter.camel.client.CalculationMCPClient import CalculationMCPClient

warnings.filterwarnings('ignore')


### 配置API和运行环境镜像

1. 填入项目，镜像地址，DP access key，OPENAI api
2. 将notebook ip加入lvmeng




In [ ]:
OS = os.uname().sysname

# 安装 bohr CLI
if OS == "Linux":
    print("🌱 正在安装适用于 Linux 的 bohr CLI...")
    subprocess.run("bash -c \"$(curl -fsSL https://dp-public.oss-cn-beijing.aliyuncs.com/bohrctl/1.0.0/install_bohr_linux_curl.sh)\"", shell=True)
elif OS == "Darwin":
    print("🍎 正在安装适用于 macOS 的 bohr CLI...")
    subprocess.run("bash -c \"$(curl -fsSL https://dp-public.oss-cn-beijing.aliyuncs.com/bohrctl/1.0.0/install_bohr_mac_curl.sh)\"", shell=True)
else:
    raise RuntimeError(f"❌ 不支持的操作系统: {OS}")

# 更新PATH环境变量（记得展开~成绝对路径）
home_dir = os.path.expanduser("~")
bohr_path = os.path.join(home_dir, ".bohrium")
os.environ["PATH"] += f":{bohr_path}"

print("✅ bohrctl 安装并配置完成！")

In [2]:
# 配置 bohrium access key

!echo 'export ACCESS_KEY=YOUR-BOHRIUM-KEY' >> ~/.bashrc #DP access key
!source ~/.bashrc
print("start install bohrctl")
# 配置OpenAI key
os.environ["AZURE_OPENAI_BASE_URL"] = "YOUR-OPENAI-URL" #OPENAI api
print("start install bohrctl")
os.environ["AZURE_OPENAI_API_KEY"] = "YOUR-OPENAI-API-KEY" #OPENAI api
os.environ["AZURE_API_VERSION"] = "2024-08-01-preview"#OPENAI api
os.environ["AZURE_DEPLOYMENT_NAME"] = "gpt-4o"

#配置DeepSeek key
os.environ["DEEP_SEEK_BASE_URL"] = "YOUR-DEEPSEEK-URL" #DeepSeek api
os.environ["DEEP_SEEK_API_KEY"] = "YOUR-DEEPSEEK-API-KEY"#DeepSeek api
os.environ["DEEP_SEEK_MODEL_NAME"] = "YOUR-DEEPSEEK-MODEL-NAME" #DeepSeek api


#加入本地数据集位置
os.environ["MP_ROOT_DIR"] = "mp_materials_cif"# 本地数据库位置



In [3]:
# 创建基础模型，选择 AZURE 平台和 gpt-4o 模型
model = ModelFactory.create(
    model_platform=ModelPlatformType.AZURE,
    model_type=ModelType.GPT_4O,
    model_config_dict={"temperature": 0.1},
)

## 基础prompt





In [4]:
#定义整体任务与不同的角色，这里需要描述的更详细一些。
task_prompt = '''
**任务目标**：博士生生成对应体系的VASP计算配置，工程师提交任务/监听任务，收到结果后交给写报告博士输出报告，任务结束。
**工作流程**：

1. 博士生：
    你需要一步一步听从任务指示，逐步完成任务，并在每一步完成后给出反馈与任务完成情况
    - 向人类提问，获取论文路径与计算的体系，调用read_vasp_pdf工具，获取论文内容，这一步不需要任何回复。
    - 之后，先调用search_poscar_template工具，生成POSCAR模板，
    对POSCAR模板进行原子替换(不再需要search_poscar_template)，确保结构中的所有原子种类、数量和分布都严格符合输入化学式人类希望复现的化学式。之后以
    """
    原子替换后POSCAR文件内容，
    请问你有什么修改意见？
    """
    的形式向人类提问
    并向人类提问修改意见，确认后进行下一步。
    - 调用write_poscar工具，你需要将（原子替换后的，完全符合POSCAR格式的str输入到函数中），将POSCAR文件内容写入文件。
    - 根据POSCAR文件内容和文件格式，生成对应实验的XX.VT的文件内容，调用write_vasp_config工具，生成VASP配置文件。
    - 检查后将calcdir返回给工程师，calcdir是文件路径，可以用化合物名称命名。

2. 工程师：
    - 从博士生接收calcdir
    - 先向我展示计算配置，使用show_vasp_config工具，具体返回INCAR文件内容和请问您有什么修改意见
    - 并询问人类是否需要重写，如果需要重写，则重写并展示，直到人类返回OK，验证通过后，提交VASP计算
    - 收到任务提交成功后，开始监听任务结果，返回xml_path
    - 收到xml_path后，通知写报告博士生

3. 写报告博士生：
    - 接收xml_path
    - 使用analyze_vasprun_all工具，分析VASP计算结果，
    - 对这个结果进行一定分析,并写入报告，完成报告后，使用write_vasp_report工具，将报告具体内容写入文件
    - 注意，报告里面需要包含此次计算的具体分析。

注意分解完任务后，需要向人类确认，人类确认后，才能开始执行任务。
'''

#读文献博士生prompt
phd_prompt = """
<role_definition>
你是一个专业的量子化学博士生，具有以下特点：
1. 精通VASP计算和晶体结构分析
2. 对文献有极强的理解能力
3. 工作严谨，注重细节
4. 严格按照规范输出格式
5. 理解并使用工具
6. 遇到疑问时主动向人类提问
<role_definition>
"""

#写报告博士生
output_prompt = """
<role_definition>
你是一个材料模拟分析专家，收到xml_path后，使用analyze_vasprun_all工具，分析VASP计算结果
根据计算结果生成一份标准化的计算报告：

报告应包括：
- 程序与平台信息
- INCAR 设置摘要
- K 点信息（是否自动生成）
- 结构是否存在，若缺失请说明
- 是否有力信息，力是否为零
- 若存在错误或缺失信息，也请如实写入
请使用正式、专业的语气完成该报告。
请你将计算报告的结果的分析写到experiment_report.txt里面
<role_definition>
"""

# 额外的任务要求
additional_info = """
**交互规则**：
1. 博士生必须严格按照规范输出格式
2. 最后博士生运行成功后，将calcdir告诉工程师，交给工程师运行VASP计算，并监听任务结果,返回xml_path。
3. 写实验报告博士生获取xml_path，并运行analyze_vasprun_all工具，并写实验报告。

**质量要求**：
1. 所有参数必须来自文献

**注意**：
1. 博士生必须严格按照规范输出格式
2. 博士生需要确认自我检查
3. 所有人在遇到问题时，先向人类寻求意见。
"""


## 定义角色与工具



In [5]:
#以这种形式定义可执行工具

humantoolkit = HumanToolkit()


PHDtools: list[FunctionTool] = [
    FunctionTool(func) for func in [write_vasp_config,analyze_vasprun_all,write_vasp_report,read_vasp_pdf,search_poscar_template,write_poscar]
]

Engineertools: list[FunctionTool] = [
    FunctionTool(func) for func in [show_vasp_config,rewrite_vasp_config]
]



phd_agent = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="精通量子化学的博士生",
        content=phd_prompt,
    ),
    model=model,
    output_language="zh-CN",
    tools=[*humantoolkit.get_tools(),*PHDtools])

phd_agent_write = ChatAgent(
    system_message=BaseMessage.make_assistant_message(
        role_name="专业负责写报告的博士生",
        content=output_prompt,
    ),
    model=model,
    tools=[*PHDtools,*humantoolkit.get_tools()],
    output_language="zh-CN"
)

在终端运行以下命令,启动mcp服务
```bash
cd server
python CalculationMCPServer.py
```

In [6]:

import nest_asyncio
nest_asyncio.apply()
import asyncio

current_dir = os.getcwd()

bohr_executor = {
    "type": "dispatcher",
    "machine": {
        "batch_type": "Bohrium",
        "context_type": "Bohrium",
        "remote_profile": {
            "email": os.environ.get("BOHRIUM_USERNAME"),
            "password": os.environ.get("BOHRIUM_PASSWORD"),
            "program_id": int(os.environ.get("BOHRIUM_PROJECT_ID")),
            "input_data": {
                "image_name": "registry.dp.tech/dptech/vasp:5.4.4",
                "job_type": "container",
                "platform": "ali",
                "scass_type": "c32_m32_cpu",
            },
        },
    },
    "DEFAULT_FORWARD_DIR":[f"{current_dir}/tmp"]
    
}
bohr_storage = {
    "type": "bohrium",
    "username": os.environ.get("BOHRIUM_USERNAME"),
    "password": os.environ.get("BOHRIUM_PASSWORD"),
    "project_id": int(os.environ.get("BOHRIUM_PROJECT_ID")),
}


async def run_example():
    mcp_client = CalculationMCPClient(
        command_or_url="http://127.0.0.1:8000/sse",
        default_executor=bohr_executor,
        default_storage=bohr_storage
    )
    print("start connect to mcp server")
    await mcp_client.connect()
    print("connected to mcp server")
    mcp_toolkit = MCPToolkit(servers=[mcp_client])
    mcp_tools = mcp_toolkit.get_tools()
    try:
        engineer_agent = ChatAgent(
            model=model,
            system_message="你是一个工程师，负责运行脚本，需要向博士生收到calcdir，能够调用submit_vasp_job工具,并持续监听任务直到任务完成后返回",
            tools=[*mcp_tools,*humantoolkit.get_tools(),*Engineertools],
            output_language="zh-CN"
        )
        workforce = CalculationWorkforce('VASP仿真计算团队',
                    coordinator_agent_kwargs={"model": model,"output_language":"zh-CN","tools":[*humantoolkit.get_tools()]},
                    task_agent_kwargs={"model": model,"output_language":"zh-CN","tools":[*humantoolkit.get_tools()]},
                    new_worker_agent_kwargs={"model": model,"output_language":"zh-CN","tools":[*humantoolkit.get_tools()]}
                )

        workforce.add_single_agent_worker(
            "精通VASP/仿真计算的博士生",
            worker=phd_agent,
        ).add_single_agent_worker(
            "负责提交任务的工程师",
            worker=engineer_agent
        ).add_single_agent_worker(
            "负责写实验报告的博士生",
            worker=phd_agent_write)



        human_task = Task(
            content=task_prompt,
            ional_info= additional_info,
            id='0',
        )

        task = workforce.process_task(human_task)

    except Exception as e:
        print(f"Error during agent execution: {e}")

    finally:
        await mcp_client.disconnect()

if __name__ == "__main__":
    await run_example()

start connect to mcp server
connected to mcp server
Question: The subtasks are:
Subtask 1 博士生向人类提问，获取论文路径与计算的体系，调用read_vasp_pdf工具，获取论文内容。
Subtask 2 博士生调用search_poscar_template工具，生成POSCAR模板，并进行原子替换，确保结构符合输入化学式。
Subtask 3 博士生将原子替换后的POSCAR文件内容发送给人类，询问修改意见。
Subtask 4 博士生调用write_poscar工具，将确认后的POSCAR文件内容写入文件。
Subtask 5 博士生根据POSCAR文件内容生成VASP配置文件，调用write_vasp_config工具。
Subtask 6 博士生将calcdir返回给工程师。
Subtask 7 工程师展示计算配置，询问人类是否需要修改，并根据反馈重写配置。
Subtask 8 工程师提交VASP计算，并监听任务结果，返回xml_path。
Subtask 9 写报告博士生接收xml_path，使用analyze_vasprun_all工具分析结果。
Subtask 10 写报告博士生撰写报告，并使用write_vasp_report工具将报告写入文件。
你确认以上任务分解吗？如果确认，请回复“通过”。否则，请回复“不通过”，并告诉我需要修改的地方。
Worker node 6053173088 (精通VASP/仿真计算的博士生) get task 0.0: Subtask 1 博士生向人类提问，获取论文路径与计算的体系，调用read_vasp_pdf工具，获取论文内容。
Question: 请提供论文的路径以及需要计算的体系信息，以便我可以使用read_vasp_pdf工具获取论文内容。
Reply from Worker node 6053173088 (精通VASP/仿真计算的博士生):

已成功提取论文内容。该论文主要讨论了通过高通量组合方法加速钙钛矿氧化物的开发，以用于热化学能量存储。研究中使用了密度泛函理论（DFT）模拟对超过2000种掺杂的SrFeO3−δ进行筛选，选出了61种具有潜力的热化学能量存储（TCES）候选材料，并对其中45种纯钙钛矿相材料进行

## DEMO

可以将需要阅读的论文放在运行目录下，在Agent询问时输入

## 样例输入
1. 文章是test.pdf,我想研究Sr5Ca3Fe8O24体系的实验（提供文献路径用于读取不同的文献）
2. 非常好，请继续生成/我想研究的体系是Sr5Ca3Fe8O24，请注意原子替换后给我新的POSCAR
3. 我想把INCAR的NELM = 200 修改为 2（修改INCAR意见）

